In [ ]:
import pandas as pd
import numpy as np
import datarobot as dr
import os
import time

In [ ]:
USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']

dr.Client(token=API_TOKEN, endpoint=ENDPOINT)  

DEPLOYMENT_ID = '5c19273c06eeed008a2ac7f9'

deployment = dr.Deployment.get(deployment_id=DEPLOYMENT_ID)

PROJECT_ID = deployment.model.get('project_id')
MODEL_ID = deployment.model.get('id')

project = dr.Project.get(project_id=PROJECT_ID)
model = dr.Model.get(project=PROJECT_ID, model_id=MODEL_ID)

In [ ]:
# Let's time it:
t1 = time.time()

PRED_FILE = 'my_prediction_rows.csv'

print('Uploading prediction dataset')
dataset_from_path = project.upload_dataset(PRED_FILE)

print('Initializing prediction explanations')
pei = dr.PredictionExplanationsInitialization.create(project.id, model.id)
print('- ', pei)

print('Waiting for initialization')
print(pei.get_result_when_complete())

print('First, request predictions')
predict_job = model.request_predictions(dataset_from_path.id)

print('Waiting for prediction calculations')
predictions = predict_job.get_result_when_complete()

print('Then you can create prediction explanations')
pe_job = dr.PredictionExplanations.create(project.id, model.id,  dataset_from_path.id)

print('Waiting for job to complete')
pe = pe_job.get_result_when_complete()

print('- Time: %0.3f' % (time.time()-t1))
df_pe = pe.get_all_as_dataframe()
df_pe.head()